In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img, img_to_array, array_to_img
from keras.datasets import mnist
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, auc, roc_curve
from tensorflow.keras.applications import InceptionResNetV2  # Use Inception-ResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout


In [7]:
# Check GPU availability
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [8]:
# Load the dataset and visualize images
labels = os.listdir("/kaggle/input/drowsiness-dataset/train")


In [9]:
#import os
directory_path = "/kaggle/working/augmented_dataset/"
os.makedirs(directory_path)
directory_path = "/kaggle/working/augmented_dataset/train"
os.makedirs(directory_path)
directory_path = "/kaggle/working/augmented_dataset/train/Closed"
os.makedirs(directory_path)
directory_path = "/kaggle/working/augmented_dataset/train/Open"
os.makedirs(directory_path)
directory_path = "/kaggle/working/augmented_dataset/train/no_yawn"
os.makedirs(directory_path)
directory_path = "/kaggle/working/augmented_dataset/train/yawn"
os.makedirs(directory_path)

In [10]:
# Closed 
# Define your dataset directory and other parameters
dataset_dir = '/kaggle/input/drowsiness-dataset'
output_dir = '/kaggle/working/augmented_dataset'
augmentation_factor = 5  # Adjust this to determine how much you want to augment your dataset

# Create an ImageDataGenerator with augmentation settings
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and augment images from the dataset
for root, _, files in os.walk(dataset_dir):
    for filename in files:
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Adjust extensions as needed
            img = load_img(os.path.join(root, filename))
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)
            
            i = 0
            for batch in datagen.flow(x, batch_size=1):
                augmented_image = array_to_img(batch[0])
                augmented_image.save(os.path.join(output_dir, f'augmented_{i}_{filename}'))
                i += 1
                if i >= augmentation_factor:
                    break

print(f'Dataset augmentation complete. Augmented {augmentation_factor} times.')


Dataset augmentation complete. Augmented 5 times.


In [13]:
# Load the dataset and visualize images
labels = os.listdir("/kaggle/working/augmented_dataset")

# Load the data using ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30
)

In [12]:
# Load the dataset and split into training and testing sets
directory = "../kaggle/working/augmented_dataset"
target_size = (240, 240)
batch_size = 4000  # Adjust batch size as needed

# Load the dataset and split into training and testing sets
data = datagen.flow_from_directory(
    directory,
    target_size=target_size,
    batch_size=batch_size,  # Specify batch size here
    class_mode='categorical',
    subset='training'
)

# Split the data into training and testing sets
x, y = data.next()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: '../kaggle/working/augmented_dataset'

In [ ]:
# Encode the labels using one-hot encoding
label_bin = LabelBinarizer()
y_train = label_bin.fit_transform(y_train)
y_test = label_bin.transform(y_test)

In [ ]:
# Create the Inception-ResNetV2 base model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(240, 240, 3))


In [ ]:
# Add custom layers for classification on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='sigmoid')
])


In [ ]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)
model.summary()


In [ ]:
# Train the model
hist = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), batch_size=45)  # Specify batch size here

In [ ]:
# Save the model
model.save("drowsiness_inceptionv4.h5")
model.save("drowsiness_inceptionv4.model")

In [ ]:
# Plot accuracy and loss
accuracy = hist.history['accuracy']
val_accuracy = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, "b", label="training accuracy")
plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training & Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss, "b", label="training loss")
plt.plot(epochs, val_loss, "r", label="validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.show()


In [ ]:
# Make predictions and evaluate the model
y_true = np.argmax(y_test, axis=1)
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)


In [ ]:
# Confusion Matrix
confusion_mtx = confusion_matrix(y_true, y_pred_classes)

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Print the confusion matrix
print("Confusion Matrix:\n", confusion_mtx)

In [ ]:
# Accuracy Score
accuracy = np.sum(y_true == y_pred_classes) / len(y_true)
print("Accuracy:", accuracy)


In [ ]:
# Compute ROC curves and AUC for each class
n_classes = 4
fpr = dict()
tpr = dict()

roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curves for each class
plt.figure(figsize=(8, 6))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], lw=2, label='Class {} (AUC = {:.2f})'.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
